In [ ]:
!pip install transformers
!git clone https://github.com/huggingface/transformers
!pip install wget

Cloning into 'transformers'...
remote: Enumerating objects: 215447, done.
remote: Counting objects: 100% (17988/17988), done.
remote: Compressing objects: 100% (1898/1898), done.
remote: Total 215447 (delta 17006), reused 16591 (delta 15988), pack-reused 197459
Receiving objects: 100% (215447/215447), 220.30 MiB | 10.78 MiB/s, done.
Resolving deltas: 100% (156437/156437), done.
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=eb5943d1ffd76238c7363a09a348f83fc052cbdd1cf8036c4d7732b55a0e4c1f
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from transformers import LxmertTokenizer, LxmertForPreTraining
import torch
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
import os
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load tokenizer and model
tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
model = LxmertForPreTraining.from_pretrained('unc-nlp/lxmert-base-uncased').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [ ]:
import sys
sys.path.append('/content/transformers/examples/research_projects/lxmert')
from utils import Config
from processing_image import Preprocess
from modeling_frcnn import GeneralizedRCNN

In [ ]:
frcnn_cfg = Config.from_pretrained('unc-nlp/frcnn-vg-finetuned')
frcnn = GeneralizedRCNN.from_pretrained('unc-nlp/frcnn-vg-finetuned', config=frcnn_cfg)
image_preprocess = Preprocess(frcnn_cfg)

%s not found in cache or force_download set to True, downloading to %s https://s3.amazonaws.com/models.huggingface.co/bert/unc-nlp/frcnn-vg-finetuned/config.yaml /root/.cache/torch/transformers/tmpr_89isx4


Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

loading configuration file cache
%s not found in cache or force_download set to True, downloading to %s https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin /root/.cache/torch/transformers/tmpjm9iqyg9


Downloading:   0%|          | 0.00/262M [00:00<?, ?B/s]

loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /root/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Project2_4/dataset.csv')

In [ ]:
def get_label_scores(image_path, labels):
  image = Image.open(image_path).convert('RGB')
  image, sizes, scales_yx = image_preprocess(image_path)
  output_dict = frcnn(image, sizes, scales_yx=scales_yx, padding ='max_detections',
                      max_detections=frcnn_cfg.max_detections, return_tensors='pt')
  normalized_boxes = output_dict.get('normalized_boxes')
  features = output_dict.get('roi_features')

  scores = {}
  for label in labels:
    encoding = tokenizer(label, truncation=True, return_token_type_ids=True,
                         return_attention_mask=True, add_special_tokens=True,
                         return_tensors="pt").to(device)
    outputs = model(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask,
                    visual_feats=features, visual_pos=normalized_boxes,
                    token_type_ids=encoding.token_type_ids, return_dict=True,
                    output_attentions=False)
    m = torch.nn.Softmax(dim=1)
    cross_score = m(outputs['cross_relationship_score'])
    scores[label] = cross_score[0][0].item()
  return scores

results = []

for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
    image_path = row['image']
    labels = [row['label1'], row['label2'], row['label3']]

    # Get scores
    scores = get_label_scores(image_path, labels)

    # Rank labels based on scores (1 = best, 3 = worst)
    ranked_labels = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    ranked_dict = {ranked_labels[i][0]: i + 1 for i in range(len(ranked_labels))}

    # Add to results
    results.append({
        'image': image_path,
        'label1': labels[0],
        'label2': labels[1],
        'label3': labels[2],
        'label1_rank': ranked_dict[labels[0]],
        'label2_rank': ranked_dict[labels[1]],
        'label3_rank': ranked_dict[labels[2]]
    })

results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/Project2_4/LXMERT_labels.csv', index=False)
results_df


  0%|          | 0/36 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


,image,label1,label2,label3,label1_rank,label2_rank,label3_rank
0,/content/drive/MyDrive/Project2_4/images/new1.png,The phone on the left is new,The phone on the left is newer,The phone on the left is the newest,2,3,1
1,/content/drive/MyDrive/Project2_4/images/new2.png,The phone on the left is new,The phone on the left is newer,The phone on the left is the newest,2,1,3
2,/content/drive/MyDrive/Project2_4/images/new3.png,The phone on the left is new,The phone on the left is newer,The phone on the left is the newest,1,2,3
3,/content/drive/MyDrive/Project2_4/images/short...,The woman's hair is short,The woman's hair is shorter,The woman's hair is the shortest,1,2,3
4,/content/drive/MyDrive/Project2_4/images/short...,The woman's hair is short,The woman's hair is shorter,The woman's hair is the shortest,1,2,3
5,/content/drive/MyDrive/Project2_4/images/short...,The woman's hair is short,The woman's hair is shorter,The woman's hair is the shortest,1,2,3
6,/content/drive/MyDrive/Project2_4/images/young...,The young boy is crying,The younger boy is crying,The youngest boy is crying,1,2,3
7,/content/drive/MyDrive/Project2_4/images/young...,The young boy is crying,The younger boy is crying,The youngest boy is crying,2,1,3
8,/content/drive/MyDrive/Project2_4/images/young...,The young boy is crying,The younger boy is crying,The youngest boy is crying,3,1,2
9,/content/drive/MyDrive/Project2_4/images/apple...,Earth is a large planet,Earth is a larger planet,Earth is the largest planet,1,3,2
